In [1]:
import cleanText
from tensorflowEval import ClassificationReport
from DisasterDetector import DisasterDetector

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [3]:
import tensorflow_hub as hub

In [7]:
df_train = pd.read_csv("../data/raw/train.csv",dtype={'id': np.int16, 'target': np.int8})
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
#df_train['text_cleaned'] = df_train['text'].apply(lambda s : cleanText.clean(s))
df_train = pd.read_csv("../data/raw/trainProc.csv",dtype={'id': np.int16, 'target': np.int8})
#df_train[["id","text_cleaned","target"]].to_csv("../data/raw/trainProc.csv",index=False)
df_train.head()

,id,text_cleaned,target
0,1,Our Deeds are the Reason of this # earthquake...,1
1,4,Forest fire near La Ronge Sask . Canada,1
2,5,All residents asked to ' shelter in place ' ...,1
3,6,"13,000 people receive # wildfires evacuation ...",1
4,7,Just got sent this photo from Ruby # Alaska a...,1


In [5]:
K = 2
seed = 100
skf = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)

In [6]:
%%time
#module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 11.5 s, sys: 3.11 s, total: 14.6 s
Wall time: 52 s


In [9]:
%%time
clf = DisasterDetector(bert_layer,skf, max_seq_length=27,lr=0.0001, 
                       epochs=10, 
                       batch_size=32)

clf.train(df_train)


Fold 0

Epoch 1/10
119/119 [==============================] - ETA: 0s - loss: 0.6842 - accuracy: 0.5888
Epoch: 1 - Training Precision: 0.737729 - Training Recall: 0.684738 - Training F1: 0.684214
Epoch: 1 - Validation Precision: 0.713395 - Validation Recall: 0.666675 - Validation F1: 0.664692
119/119 [==============================] - 701s 6s/step - loss: 0.6842 - accuracy: 0.5888 - val_loss: 0.6146 - val_accuracy: 0.6974
Epoch 2/10
119/119 [==============================] - ETA: 0s - loss: 0.5633 - accuracy: 0.7488
Epoch: 2 - Training Precision: 0.765834 - Training Recall: 0.756316 - Training F1: 0.759263
Epoch: 2 - Validation Precision: 0.743897 - Validation Recall: 0.733994 - Validation F1: 0.736788
119/119 [==============================] - 907s 8s/step - loss: 0.5633 - accuracy: 0.7488 - val_loss: 0.5391 - val_accuracy: 0.7465
Epoch 3/10
119/119 [==============================] - ETA: 0s - loss: 0.4944 - accuracy: 0.7801
Epoch: 3 - Training Precision: 0.793898 - Training Recall: 

CPU times: user 21h 7min 19s, sys: 2h 36min 55s, total: 23h 44min 14s
Wall time: 4h 25min 38s


## Explore details of BERT layer

In [11]:
from tokenization import load_vocab

In [7]:
blVocabFile = bert_layer.resolved_object.vocab_file.asset_path.numpy()

In [12]:
properVocab = load_vocab(blVocabFile)

In [13]:
type(properVocab)

collections.OrderedDict

In [25]:
tokenInBert = list(properVocab.keys())

In [28]:
len(tokenInBert)

30522

In [75]:
#english tonkens ranges from
start = 1996
end = 29611
print(tokenInBert[start],tokenInBert[end])
print("Number of english tokens",(end-start))

the necessitated
Number of english tokens 27615


In [27]:
tokenInBert[200]

'[unused195]'

In [23]:
properVocab.keys()

TypeError: 'odict_keys' object is not subscriptable

In [19]:
import sys
sys.getsizeof(properVocab)/10**6

2.81188

In [18]:
?sys.getsizeof

In [77]:
texts = list(df_train.text_cleaned)

In [89]:
text = texts[0]

In [ ]:
from tokenization import FullTokenizer
tokenizer = FullTokenizer(blVocabFile, True)

In [97]:
text = 'Our Deeds are the,Reason of this  # earthquake May ALLAH Forgive us all'

In [98]:

text = tokenizer.tokenize(text)
text

['our',
 'deeds',
 'are',
 'the',
 ',',
 'reason',
 'of',
 'this',
 '#',
 'earthquake',
 'may',
 'allah',
 'forgive',
 'us',
 'all']

In [118]:
tokenizer.tokenize("arpan")

['ar', '##pan']

In [100]:
max_seq_length = 27
text = text[:max_seq_length - 2]
text

['our',
 'deeds',
 'are',
 'the',
 ',',
 'reason',
 'of',
 'this',
 '#',
 'earthquake',
 'may',
 'allah',
 'forgive',
 'us',
 'all']

In [101]:
input_sequence = ['[CLS]'] + text + ['[SEP]']
input_sequence

['[CLS]',
 'our',
 'deeds',
 'are',
 'the',
 ',',
 'reason',
 'of',
 'this',
 '#',
 'earthquake',
 'may',
 'allah',
 'forgive',
 'us',
 'all',
 '[SEP]']

In [ ]:
Mg

In [131]:
for x in range(0,1000):
    print(x,properVocab[str(x)])

0 1014
1 1015
2 1016
3 1017
4 1018
5 1019
6 1020
7 1021
8 1022
9 1023
10 2184
11 2340
12 2260
13 2410
14 2403
15 2321
16 2385
17 2459
18 2324
19 2539
20 2322
21 2538
22 2570
23 2603
24 2484
25 2423
26 2656
27 2676
28 2654
29 2756
30 2382
31 2861
32 3590
33 3943
34 4090
35 3486
36 4029
37 4261
38 4229
39 4464
40 2871
41 4601
42 4413
43 4724
44 4008
45 3429
46 4805
47 4700
48 4466
49 4749
50 2753
51 4868
52 4720
53 5187
54 5139
55 4583
56 5179
57 5401
58 5388
59 5354
60 3438
61 6079
62 5786
63 6191
64 4185
65 3515
66 5764
67 6163
68 6273
69 6353
70 3963
71 6390
72 5824
73 6421
74 6356
75 4293
76 6146
77 6255
78 6275
79 6535
80 3770
81 6282
82 6445
83 6640
84 6391
85 5594
86 6564
87 6584
88 6070
89 6486
90 3938
91 6205
92 6227
93 6109
94 6365
95 5345
96 5986
97 5989
98 5818
99 5585
100 2531
101 7886
102 9402
103 9800
104 9645
105 8746
106 10114
107 10550
108 10715
109 11518
110 7287
111 11118
112 11176
113 12104
114 12457
115 10630
116 12904
117 12567
118 12963
119 13285
120 6036
121 1260

KeyError: '342'

In [125]:
for x in range(97,123):
    print(chr(x),properVocab[chr(x)])

a 1037
b 1038
c 1039
d 1040
e 1041
f 1042
g 1043
h 1044
i 1045
j 1046
k 1047
l 1048
m 1049
n 1050
o 1051
p 1052
q 1053
r 1054
s 1055
t 1056
u 1057
v 1058
w 1059
x 1060
y 1061
z 1062


In [117]:
tokenizer.convert_tokens_to_ids(["hi","him"])

[7632, 2032]

In [102]:
tokens = tokenizer.convert_tokens_to_ids(input_sequence)
tokens

[101,
 2256,
 15616,
 2024,
 1996,
 1010,
 3114,
 1997,
 2023,
 1001,
 8372,
 2089,
 16455,
 9641,
 2149,
 2035,
 102]

In [ ]:




pad_len = self.max_seq_length - len(input_sequence)
tokens += [0] * pad_len
pad_masks = [1] * len(input_sequence) + [0] * pad_len
segment_ids = [0] * self.max_seq_length

all_tokens.append(tokens)
all_masks.append(pad_masks)
all_segments.append(segment_ids

In [ ]:
all_tokens = []
all_masks = []
all_segments = []

for text in texts:
    text = self.tokenizer.tokenize(text)
    text = text[:self.max_seq_length - 2]
    input_sequence = ['[CLS]'] + text + ['[SEP]']
    pad_len = self.max_seq_length - len(input_sequence)

    tokens = self.tokenizer.convert_tokens_to_ids(input_sequence)
    tokens += [0] * pad_len
    pad_masks = [1] * len(input_sequence) + [0] * pad_len
    segment_ids = [0] * self.max_seq_length

    all_tokens.append(tokens)
    all_masks.append(pad_masks)
    all_segments.append(segment_ids)

return np.array(all_tokens), np.array(all_masks), np.array(all_segments)